In [ ]:
import pandas as pd
import dalmatian as dm
from depmapomics.config import *

In [ ]:
def generateSigTable(refworkspace,
    todrop=[],
    priority=[],
    purecncols=SIGTABLE_TERRACOLS,
    misccols=MISC_SIG_TERRACOLS,
    purecn_failed_col=PURECN_FAILED_COLNAME,
    binary_cols=SIGTABLE_BINARYCOLS,
    colrenaming=SIGTABLE_RENAMING,
    save_output="",
):
    wm = dm.WorkspaceManager(refworkspace).disable_hound()
    samples = wm.get_samples()
    samples = samples[samples.index != 'nan']
    
    # discard lines that have failed PureCN for the PureCN columns
    purecn_passed = samples[samples[purecn_failed_col] == 'FALSE']
    purecn_table = purecn_passed[purecncols]
    purecn_table = purecn_table[~purecn_table.index.isin(set(todrop) - set(priority))]
    
    # use 1/0 for binary values
    purecn_table = purecn_table.replace({'TRUE': 1, 'FALSE': 0})
    purecn_table[binary_cols] = purecn_table[binary_cols].astype('Int64')
    
    # table containing cols that are not generated by pureCN,
    # which may include samples that have failed pureCN
    misc_table = samples[misccols]
    sig_table = misc_table.merge(purecn_table, how='outer', left_index=True, right_index=True)
    
    # rename columns
    sig_table = sig_table.rename(columns=colrenaming)
    
    print("PureCN: saving files")
    #sig_table.to_csv(save_output + "cell_line_signature_table_all.csv")
    print("done")
    
    return sig_table

In [ ]:
from depmapomics import copynumbers as cn

wes_table = cn.generateSigTable("broad-firecloud-ccle/DepMap_WES_CN_hg38")
wes_table